In [1]:
X=[1 2 3 1 2 3 1 2 3;1 1 1 2 2 2 3 3 3]

2×9 Array{Int64,2}:
 1  2  3  1  2  3  1  2  3
 1  1  1  2  2  2  3  3  3

In [13]:
X[:,3][1]

3

In [2]:
f(x)=[sqrt(x[1]), x[2]^2, x[1]+x[2]]

f (generic function with 1 method)

In [3]:
Y=zeros(3,9)
for i in 1:9
    Y[:,i]=f(X[:,i])
end
Y

3×9 Array{Float64,2}:
 1.0  1.41421  1.73205  1.0  1.41421  1.73205  1.0  1.41421  1.73205
 1.0  1.0      1.0      4.0  4.0      4.0      9.0  9.0      9.0
 2.0  3.0      4.0      3.0  4.0      5.0      4.0  5.0      6.0

## Choice of two types of Radial Basis Function :
* Linear spline (G1)
* Cubic spline (G2)

In [16]:
G1=zeros(9,9)
for i in 1:9
    for j in 1:9
    G1[i,j]=sqrt((X[:,i][1]-X[:,j][1])^2+(X[:,i][2]-X[:,j][2])^2)
    end
end
G1

9×9 Array{Float64,2}:
 0.0      1.0      2.0      1.0      …  2.23607  2.0      2.23607  2.82843
 1.0      0.0      1.0      1.41421     1.41421  2.23607  2.0      2.23607
 2.0      1.0      0.0      2.23607     1.0      2.82843  2.23607  2.0
 1.0      1.41421  2.23607  0.0         2.0      1.0      1.41421  2.23607
 1.41421  1.0      1.41421  1.0         1.0      1.41421  1.0      1.41421
 2.23607  1.41421  1.0      2.0      …  0.0      2.23607  1.41421  1.0
 2.0      2.23607  2.82843  1.0         2.23607  0.0      1.0      2.0
 2.23607  2.0      2.23607  1.41421     1.41421  1.0      0.0      1.0
 2.82843  2.23607  2.0      2.23607     1.0      2.0      1.0      0.0

In [17]:
G2=zeros(9,9)
for i in 1:9
    for j in 1:9
    G2[i,j]=sqrt((X[:,i][1]-X[:,j][1])^2+(X[:,i][2]-X[:,j][2])^2)^3
    end
end
G2

9×9 Array{Float64,2}:
  0.0       1.0       8.0       1.0      …   8.0      11.1803   22.6274
  1.0       0.0       1.0       2.82843     11.1803    8.0      11.1803
  8.0       1.0       0.0      11.1803      22.6274   11.1803    8.0
  1.0       2.82843  11.1803    0.0          1.0       2.82843  11.1803
  2.82843   1.0       2.82843   1.0          2.82843   1.0       2.82843
 11.1803    2.82843   1.0       8.0      …  11.1803    2.82843   1.0
  8.0      11.1803   22.6274    1.0          0.0       1.0       8.0
 11.1803    8.0      11.1803    2.82843      1.0       0.0       1.0
 22.6274   11.1803    8.0      11.1803       8.0       1.0       0.0